# SQL Ground Truth Validation

This notebook validates that every SQL in `data/data_finetune.csv` executes on the TPC-DS DuckDB and returns a result.


In [57]:
from pathlib import Path
import time
import re

import duckdb
import pandas as pd


def find_repo_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / "research_pipeline").exists():
            return p
    return start


REPO_ROOT = find_repo_root(Path.cwd())
CSV_PATH = REPO_ROOT / "research_pipeline" / "data" / "data_finetune.csv"
DB_PATH = REPO_ROOT / "research_pipeline" / "data" / "ecommerce_dw.duckdb"
OUTPUT_PATH = REPO_ROOT / "research_pipeline" / "sql_ground_truth_validation.csv"

AUTO_SETUP_DB = True
SETUP_SCALE_FACTOR = 1
FORCE_RECREATE_DB = False

FETCH_LIMIT = 50
ALLOW_MUTATION = False

print(f"Repo root: {REPO_ROOT}")
print(f"CSV path: {CSV_PATH}")
print(f"DB path: {DB_PATH}")


Repo root: /home/ubuntu/DataScience/Capstone-NLUS-VDD
CSV path: /home/ubuntu/DataScience/Capstone-NLUS-VDD/research_pipeline/data/data_finetune.csv
DB path: /home/ubuntu/DataScience/Capstone-NLUS-VDD/research_pipeline/data/ecommerce_dw.duckdb


In [58]:
def setup_tpcds_db(db_path: Path, scale_factor: int = 1, force_recreate: bool = False) -> None:
    db_path.parent.mkdir(parents=True, exist_ok=True)
    con = duckdb.connect(str(db_path))
    try:
        con.execute("INSTALL tpcds;")
        con.execute("LOAD tpcds;")

        tables = [r[0] for r in con.execute("SHOW TABLES").fetchall()]
        if tables and not force_recreate:
            print(f"Found {len(tables)} tables. Skip generation.")
            return

        if force_recreate and tables:
            for t in tables:
                con.execute(f"DROP TABLE {t}")

        print(f"Generating TPC-DS (sf={scale_factor})...")
        start = time.time()
        con.execute(f"CALL dsdgen(sf={scale_factor});")
        print(f"Data generation completed in {time.time() - start:.2f}s")
    finally:
        con.close()


if not DB_PATH.exists():
    if AUTO_SETUP_DB:
        setup_tpcds_db(DB_PATH, scale_factor=SETUP_SCALE_FACTOR, force_recreate=FORCE_RECREATE_DB)
    else:
        raise FileNotFoundError(f"TPC-DS DuckDB not found: {DB_PATH}")


In [59]:
df = pd.read_csv(CSV_PATH)
required_cols = {"ID", "Transcription", "SQL Ground Truth"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing columns: {sorted(missing)}")

print(f"Loaded {len(df)} rows.")
df.head()


Loaded 3163 rows.


,ID,Transcription,SQL Ground Truth
0,1,Lấy thông tin định danh và họ tên khách hàng,"SELECT c_customer_id, c_first_name, c_last_nam..."
1,3,Đếm tổng số lượng khách hàng trong bảng,SELECT COUNT(*) AS total_customers \nFROM cust...
2,4,Lọc danh sách khách hàng thân thiết,"SELECT c_first_name, c_last_name, c_email_addr..."
3,5,Lấy ra danh sách các khách hàng hiện tại không...,"SELECT c_first_name, c_last_name, c_email_addr..."
4,6,Đếm tổng tất cả khách hàng đến từ Mỹ,"SELECT COUNT(*) AS ""TOTAL CUSTOMER"" \r\nFROM c..."


In [60]:
DML_PATTERN = re.compile(r"\b(create|insert|update|delete|drop|alter|copy|export)\b", re.IGNORECASE)


def normalize_sql(value) -> str | None:
    if value is None or (isinstance(value, float) and pd.isna(value)):
        return None
    sql = str(value).strip()
    if not sql:
        return None
    while sql.endswith(";"):
        sql = sql[:-1].strip()
    return sql


def is_read_only(sql: str) -> bool:
    return DML_PATTERN.search(sql) is None


def execute_sql(con: duckdb.DuckDBPyConnection, sql: str) -> dict:
    start = time.time()
    try:
        cur = con.execute(sql)
        rows = cur.fetchmany(FETCH_LIMIT)
        cols = [d[0] for d in cur.description] if cur.description else []
        return {
            "exec_ok": True,
            "has_rows": len(rows) > 0,
            "row_count_sample": len(rows),
            "col_count": len(cols),
            "error_type": None,
            "error_message": None,
            "exec_time_sec": time.time() - start,
        }
    except Exception as exc:
        message = str(exc)
        error_type = message.split(":", 1)[0] if ":" in message else type(exc).__name__
        return {
            "exec_ok": False,
            "has_rows": False,
            "row_count_sample": 0,
            "col_count": 0,
            "error_type": error_type,
            "error_message": message,
            "exec_time_sec": time.time() - start,
        }


results = []
con = duckdb.connect(str(DB_PATH), read_only=True)
try:
    iterator = df.iterrows()
    try:
        from tqdm.auto import tqdm
        iterator = tqdm(iterator, total=len(df))
    except Exception:
        pass

    for _, row in iterator:
        sql_raw = row["SQL Ground Truth"]
        sql_clean = normalize_sql(sql_raw)

        record = {
            "id": row["ID"],
            "question": row["Transcription"],
            "sql_raw": sql_raw,
            "sql_clean": sql_clean,
        }

        if not sql_clean:
            record.update({
                "exec_ok": False,
                "has_rows": False,
                "row_count_sample": 0,
                "col_count": 0,
                "error_type": "EMPTY_SQL",
                "error_message": "Missing SQL Ground Truth",
                "exec_time_sec": 0.0,
            })
            results.append(record)
            continue

        if not ALLOW_MUTATION and not is_read_only(sql_clean):
            record.update({
                "exec_ok": False,
                "has_rows": False,
                "row_count_sample": 0,
                "col_count": 0,
                "error_type": "NON_READ_ONLY",
                "error_message": "Mutation statements are not allowed",
                "exec_time_sec": 0.0,
            })
            results.append(record)
            continue

        record.update(execute_sql(con, sql_clean))
        results.append(record)
finally:
    con.close()

results_df = pd.DataFrame(results)
results_df.head()


100%|██████████| 3163/3163 [00:46<00:00, 68.18it/s] 


,id,question,sql_raw,sql_clean,exec_ok,has_rows,row_count_sample,col_count,error_type,error_message,exec_time_sec
0,1,Lấy thông tin định danh và họ tên khách hàng,"SELECT c_customer_id, c_first_name, c_last_nam...","SELECT c_customer_id, c_first_name, c_last_nam...",True,True,50,3,None,None,0.003689
1,3,Đếm tổng số lượng khách hàng trong bảng,SELECT COUNT(*) AS total_customers \nFROM cust...,SELECT COUNT(*) AS total_customers \nFROM cust...,True,True,1,1,None,None,0.000577
2,4,Lọc danh sách khách hàng thân thiết,"SELECT c_first_name, c_last_name, c_email_addr...","SELECT c_first_name, c_last_name, c_email_addr...",True,True,50,3,None,None,0.003914
3,5,Lấy ra danh sách các khách hàng hiện tại không...,"SELECT c_first_name, c_last_name, c_email_addr...","SELECT c_first_name, c_last_name, c_email_addr...",True,True,50,3,None,None,0.002630
4,6,Đếm tổng tất cả khách hàng đến từ Mỹ,"SELECT COUNT(*) AS ""TOTAL CUSTOMER"" \r\nFROM c...","SELECT COUNT(*) AS ""TOTAL CUSTOMER"" \r\nFROM c...",True,True,1,1,None,None,0.002503


In [61]:
total = len(results_df)
exec_ok = results_df["exec_ok"].sum()
non_empty = results_df["has_rows"].sum()
empty_sql = (results_df["error_type"] == "EMPTY_SQL").sum()

print(f"Total queries: {total}")
print(f"Exec OK: {exec_ok} ({exec_ok / total:.2%})")
print(f"Has rows (sample): {non_empty} ({non_empty / total:.2%})")
print(f"Empty SQL: {empty_sql}")

error_counts = results_df.loc[~results_df["exec_ok"], "error_type"].value_counts()
error_counts


Total queries: 3163
Exec OK: 3163 (100.00%)
Has rows (sample): 3163 (100.00%)
Empty SQL: 0


Series([], Name: count, dtype: int64)

## Analysis

Review error rows, empty results, SQL patterns, and table coverage.


In [62]:
errors_df = results_df.loc[~results_df["exec_ok"], ["id", "question", "sql_clean", "error_type", "error_message"]]
errors_df


,id,question,sql_clean,error_type,error_message


In [63]:
empty_df = results_df.loc[(results_df["exec_ok"]) & (~results_df["has_rows"]), ["id", "question", "sql_clean"]]
empty_df.head(20)


,id,question,sql_clean


In [64]:
sql_series = results_df["sql_clean"].fillna("")
features = pd.DataFrame({
    "sql_len_chars": sql_series.str.len(),
    "sql_len_tokens": sql_series.str.split().str.len(),
    "has_join": sql_series.str.contains(r"\bjoin\b", case=False, regex=True),
    "has_group_by": sql_series.str.contains(r"\bgroup\s+by\b", case=False, regex=True),
    "has_order_by": sql_series.str.contains(r"\border\s+by\b", case=False, regex=True),
    "has_limit": sql_series.str.contains(r"\blimit\b", case=False, regex=True),
})
features.describe()


,sql_len_chars,sql_len_tokens
count,3163.000000,3163.000000
mean,551.186848,65.142270
std,249.347927,29.459231
min,44.000000,5.000000
25%,428.500000,47.000000
50%,551.000000,64.000000
75%,735.000000,85.000000
max,1586.000000,189.000000


In [65]:
TABLE_PATTERN = re.compile(r"(?:from|join)\s+([a-zA-Z_][\w]*)", re.IGNORECASE)

def extract_tables(sql: str) -> list[str]:
    if not sql:
        return []
    return [t.lower() for t in TABLE_PATTERN.findall(sql)]

table_counts = sql_series.apply(extract_tables).explode().value_counts()
table_counts


sql_clean
date_dim                  2837
customer                  2265
customer_address          1941
store_sales               1147
item                      1118
web_sales                 1052
catalog_sales             1051
cust_sales                 887
store                      445
agg                        392
web_site                   348
customer_demographics      343
store_returns              257
household_demographics     178
monthly                    159
reason                      99
web_page                    80
call_center                 74
ship_mode                   64
catalog_page                52
warehouse                   49
catalog_returns             42
current_year                36
web_channel                 31
catalog_channel             31
store_channel               31
time_dim                    30
web_returns                 29
web_profit                  28
store_profit                28
catalog_profit              28
category_state_returns      2

In [66]:
dup_mask = results_df["sql_clean"].duplicated(keep=False)
duplicates = results_df.loc[dup_mask, ["id", "question", "sql_clean"]].sort_values("sql_clean")
duplicates.head(20)


,id,question,sql_clean
282,342,"Năm 1999, doanh thu từ kênh Store cao hơn hay ...",WITH store_rev AS (\n SELECT 'Store' as cha...
449,526,"Năm 1999, doanh thu từ kênh Web cao hơn hay th...",WITH store_rev AS (\n SELECT 'Store' as cha...
330,395,"Năm 2002, doanh thu từ kênh Web cao hơn hay th...",WITH store_rev AS (\n SELECT 'Store' as cha...
453,530,"Năm 2002, doanh thu từ kênh Catalog cao hơn ha...",WITH store_rev AS (\n SELECT 'Store' as cha...
494,574,"Năm 2002, doanh thu từ kênh Store cao hơn hay ...",WITH store_rev AS (\n SELECT 'Store' as cha...
569,662,"Năm 1999, doanh thu từ kênh Store cao hơn hay ...",WITH store_rev AS (\n SELECT 'Store' as chann...
660,763,"Năm 1999, doanh thu từ kênh Catalog cao hơn ha...",WITH store_rev AS (\n SELECT 'Store' as chann...
681,786,"Năm 1999, doanh thu từ kênh Store cao hơn hay ...",WITH store_rev AS (\n SELECT 'Store' as chann...
779,890,"Năm 1999, doanh thu từ kênh Catalog cao hơn ha...",WITH store_rev AS (\n SELECT 'Store' as chann...
578,673,"Năm 2000, doanh thu từ kênh Store cao hơn hay ...",WITH store_rev AS (\n SELECT 'Store' as chann...


In [67]:
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
results_df.to_csv(OUTPUT_PATH, index=False)
print(f"Saved validation results to: {OUTPUT_PATH}")


Saved validation results to: /home/ubuntu/DataScience/Capstone-NLUS-VDD/research_pipeline/sql_ground_truth_validation.csv
